In [1]:
import numpy as np
import pyvista as pv
import trimesh as tm
import os 
import copy

angles = [180] + [0] * 2 + [180] * 2

axes = [[0,1,0]] * 3 + [[0,0,1]] * 2

moves = np.array([[ 0, 0, 0],
                  [ 1, 0, 0],
                  [-1, 0, 0],
                  [ 0, 1, 0],
                  [ 0,-1, 0]]) * 2.0
sides = []
for i, a, axis, move in zip(range(5),angles, axes, moves):
    mesh_path = os.path.relpath("data/side_0" + str(i) + ".ply")
    mesh = tm.load_mesh(mesh_path)

    #rotation
    Ry = tm.transformations.rotation_matrix(np.radians(a), axis)
    mesh.apply_transform(Ry)

    # move
    mesh.vertices += move

    sides.append(mesh)

# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

class Routine():
    def __init__(self, mesh, mesh_list):
        self.output = mesh # Expected PyVista mesh type
        self.output_separate = mesh_list
        # default parameters
        self.kwargs = {
            'rot_l': 0,
            'rot_r': 0,
            'rot_u': 0,
            'rot_d': 0
        }

    def __call__(self, param, value):
        self.kwargs[param] = value
        self.update(param)

    def active(self, param, active_param):
        if param == active_param:
            return self.kwargs[param]
        else:
            return 0.0

    def update(self, param):
        # This is where you call your simulation
        R = []
        R_0 = tm.transformations.rotation_matrix(np.radians(0.0), [0,1,0], [ 1, 0, 0])
        R_l = tm.transformations.rotation_matrix(np.radians(self.active("rot_l", param)), [0,1,0], [ 1, 0, 0])
        R_r = tm.transformations.rotation_matrix(np.radians(self.active("rot_r", param)), [0,1,0], [-1, 0, 0])
        R_u = tm.transformations.rotation_matrix(np.radians(self.active("rot_u", param)), [1,0,0], [ 0, 1, 0])
        R_d = tm.transformations.rotation_matrix(np.radians(self.active("rot_d", param)), [1,0,0], [ 0,-1, 0])
        R = [R_0, R_l, R_r, R_u, R_d]
        
        new_pv_sides = []
        params = [  "rot",
                    "rot_l",
                    "rot_r",
                    "rot_u",
                    "rot_d"]

        for i, mesh, r, p in zip(range(5), sides, R, params):
            if self.active(p, param) != 0.0:
                n_mesh = copy.deepcopy(mesh)
                # rotation
                n_mesh.apply_transform(r)
                # conversion 
                pv_mesh = tri_to_pv(n_mesh)
                # append 
                new_pv_sides.append(pv_mesh)
            else:
                new_pv_sides.append(self.output_separate[i])


        all_mesh = new_pv_sides[0] + new_pv_sides[1] + new_pv_sides[2] + new_pv_sides[3] + new_pv_sides[4]

        all_mesh.point_arrays["colors"] = all_mesh_scalars

        self.output_separate = new_pv_sides
        self.output.overwrite(all_mesh)

        return


pv_sides = []
mesh_scalaras = []
for mesh in sides:
    pv_mesh = tri_to_pv(mesh)
    pv_sides.append(pv_mesh)
    mesh_scalaras.append(mesh.visual.vertex_colors)

all_mesh = pv_sides[0] + pv_sides[1] + pv_sides[2] + pv_sides[3] + pv_sides[4]
all_mesh_scalars =  np.concatenate(tuple(mesh_scalaras), axis=0)
all_mesh.point_arrays["colors"] = all_mesh_scalars


engine = Routine(all_mesh, pv_sides)

p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(all_mesh, scalars="colors", rgb=True)

p.add_slider_widget(
    callback=lambda value: engine('rot_l', value),
    rng=[-180, 180],
    value=0,
    title="Zephyrus",
    pointa=(.025, .075), pointb=(.30, .075),
    style='classic',
    event_type="always"
)
p.add_slider_widget(
    callback=lambda value: engine('rot_r', value),
    rng=[-180, 180],
    value=0,
    title="Eurus",
    pointa=(.025, .2), pointb=(.30, .2),
    style='classic',
    event_type="always"
)
p.add_slider_widget(
    callback=lambda value: engine('rot_u', value),
    rng=[-180, 180],
    value=0,
    title="Boreas",
    pointa=(.025, .325), pointb=(.30, .325),
    style='classic',
    event_type="always"
)
p.add_slider_widget(
    callback=lambda value: engine('rot_d', value),
    rng=[-180, 180],
    value=0,
    title="Notos",
    pointa=(.025, .45), pointb=(.30, .45),
    style='classic',
    event_type="always"
)

p.set_position([0,0,-10])
p.reset_camera()
p.set_background("#000000")
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(0.034177221342122865, -0.0009799664626379883, -15.833283312554004),
 (-0.05859002470970154, 0.0016799569129943848, 3.6497318709445835e-24),
 (0.0, 1.0, 0.0)]